# Лабораторная работа 1. Pandas, метод ближайших соседей и решающие деревья.

ФИО: Мерцалов А.Д.

Группа: 317

In [1]:
import numpy as np
import pandas as pd

## Часть 1: Pandas

#### Ответьте на вопросы о данных по авиарейсам в США за январь-апрель 2008 года.

Данные: http://stat-computing.org/dataexpo/2009/2008.csv.bz2
(обратите внимание, что распаковывать этот файл не обязательно — функция `pandas.read_csv` умеет читать из архивов автоматически)

Описание: http://stat-computing.org/dataexpo/2009/the-data.html

1. Какая из причин отмены рейса (`CancellationCode`) была самой частой? (расшифровки кодов можно найти в описании данных)
2. Найдите среднее, минимальное и максимальное расстояние, пройденное самолетом.
3. Не выглядит ли подозрительным минимальное пройденное расстояние? В какие дни и на каких рейсах оно было? Какое расстояние было пройдено этими же рейсами в другие дни?
4. Из какого аэропорта было произведено больше всего вылетов? В каком городе он находится?
5. Найдите для каждого аэропорта среднее время полета (`AirTime`) по всем вылетевшим из него рейсам. Какой аэропорт имеет наибольшее значение этого показателя?
6. Найдите аэропорт, у которого наибольшая доля задержанных (`DepDelay > 0`) рейсов. Исключите при этом из рассмотрения аэропорты, из которых было отправлено меньше 1000 рейсов (используйте функцию `filter` после `groupby`).

# Ответы

    1) Самая частая причина отмены рейса: weather(54904 штук)
    
    2) Min: 11, Mean: 726.387029425, Max: 4962
    
    3) Выглядит подозрительно. Эти рейсы были выполнены 15.05.2008, рейс: 4988 и 10.08.2008, рейс: 5572. Чтобы посмотреть какие расстояния были пройдены этими рейсами в другие дни см. TASK 3 в pandas.ipynb
    
    4) Аэропорт из которого вылетало больше всего самолетов: William B Hartsfield-Atlanta Intl USA 
    
    5) Среднее время полета по всем вылетевшим из него самолетам: см. TASK 5 в pandas.ipynb. Аэропорт с самым большим показателем времени полета: Luis Munoz Marin International (203.850)
    
    6) Аэропорт у которого наибольшая доля задержанных рейсов: Dallas Love. Были исключены у которых было меньше 1000 рейсов.

## Код

In [ ]:
import numpy as np
import pandas as pd
import math
import copy

data = pd.read_csv("./2008.csv.bz2", compression='bz2')


In [ ]:
#TASK 1
real_reason = {'A': 'carrier', 'B': 'weather', 'C': 'NAS', 'D': 'security'}
def get_the_most_frequent(input_array):
    buf = copy.deepcopy(input_array)
    buf = buf[buf == buf]
    buf = np.sort(buf)
    
    Y = np.concatenate((buf[1:], np.array([np.nan])))
    lens_seg = np.cumsum(np.ones(buf.shape[0]))
    val = buf[buf != Y]
    lens_seg = lens_seg[buf != Y]
    lens_seg = np.asarray(np.concatenate((lens_seg[0:1], np.diff(lens_seg))), dtype = int)
    ind = np.argmax(lens_seg)
    max = lens_seg[ind]
    max_value = val[ind]
    return (max, max_value)

column_cancell = data['CancellationCode']
(max, max_reason) = get_the_most_frequent(column_cancell)
        
print("COUNT:", max)
print("REASON:", real_reason[max_reason])

In [ ]:
#TASK 2
column_distance = data['Distance']
column_distance = column_distance[column_distance == column_distance]
min_distance = np.min(column_distance)
mean_distance = np.mean(column_distance)
max_distance = np.max(column_distance)

print("MIN:", min_distance)
print("MEAN:", mean_distance)
print("MAX:", max_distance)

In [ ]:
#TASK 3
fly_with_min_dist = data[data['Distance'] == min_distance]

idn_flight = []
for i in range(fly_with_min_dist.shape[0]):
    print(fly_with_min_dist.iloc[i][0:3])
    print(fly_with_min_dist.iloc[i]['FlightNum'])
    idn_flight.append(fly_with_min_dist.iloc[i]['FlightNum'])
    print()

#print(name_plane)

print(data[(data['FlightNum'] == idn_flight[0]) | (data['FlightNum'] == idn_flight[1])])

In [ ]:
airports = pd.read_csv("./airports.csv")
#TASK 4
origin = data['Origin']
(cnt, idn) = get_the_most_frequent(origin)
print(airports[airports['iata'] == idn]['airport'])
print(airports[airports['iata'] == idn]['country'])

In [ ]:
data_with_origin_airports = pd.merge(data, airports, left_on = 'Origin', right_on='iata')
#TASK 5
stat_by_airports = data_with_origin_airports.groupby(by='airport').mean()
print(np.argmax(stat_by_airports['AirTime']))
print(np.max(stat_by_airports['AirTime']))
print()
print(stat_by_airports['AirTime'])

In [ ]:
#TASK 6
data_with_origin_airports['count'] = 1
stat_by_airports = data_with_origin_airports.groupby(by='airport').agg({'DepDelay': (lambda x: x[x > 0].shape[0]),
                                                                      'count': np.sum})
filter_stat = stat_by_airports[stat_by_airports['count'] >= 1000]
print(np.argmax(filter_stat['DepDelay'] / filter_stat['count']))

---

## Часть 2: метрические методы и категориальные признаки

Все дальнейшие эксперименты предлагается проводить на данных соревнования Amazon Employee Access Challenge: https://www.kaggle.com/c/amazon-employee-access-challenge

В данной задаче предлагается предсказать, будет ли одобрен запрос сотрудника на получение доступа к тому или иному ресурсу. Все признаки являются категориальными.

Для удобства данные можно загрузить по ссылке: https://www.dropbox.com/s/q6fbs1vvhd5kvek/amazon.csv

Сразу прочитаем данные и создадим разбиение на обучение и контроль:

In [2]:
data = pd.read_csv('amazon.csv')
data.head()

NameError: name 'pd' is not defined

In [3]:
data.shape

(32769, 10)

In [4]:
# доля положительных примеров
data.ACTION.mean()

0.94210992096188473

In [5]:
# число значений у признаков
for col_name in data.columns:
    print (col_name, len(data[col_name].unique()))

ACTION 2
RESOURCE 7518
MGR_ID 4243
ROLE_ROLLUP_1 128
ROLE_ROLLUP_2 177
ROLE_DEPTNAME 449
ROLE_TITLE 343
ROLE_FAMILY_DESC 2358
ROLE_FAMILY 67
ROLE_CODE 343


In [6]:
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data.iloc[:, 1:], data.iloc[:, 0],
                                                    test_size=0.3, random_state=241)

/usr/local/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


#### 1. Реализуйте три функции расстояния на категориальных признаках, которые обсуждались на втором семинаре.

Проще всего будет определить метрики как [user-defined distance](http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.DistanceMetric.html), после чего воспользоваться реализацией kNN из sklearn (в этом случае используйте функцию predict_proba). Можно реализовать метод k ближайших соседей и самостоятально — в этом случае учитите, что он должен возвращать оценку вероятности, то есть отношение объектов первого класса среди соседей к числу соседей).

Постарайтесь уделить особое внимание эффективности кода — при реализации метрик "в лоб" вы можете столкнуться с очень большим временем выполнения.

#### Подсчитайте для каждой из метрик качество на тестовой выборке `X_test` при числе соседей $k = 10$. Мера качества — AUC-ROC.

#### Какая функция расстояния оказалась лучшей?

# Ответы

Реализация функций расстояния:

In [3]:
def indicator(x, y, **kwargs):
    return (np.sum(x != y))

def smoothed_indicator(x, y, **kwargs):
    buf = kwargs['p_2']
    return np.sum((x != y) + (x == y) * buf[:, np.asarray(x, int)])

def log_indicator(x, y, **kwargs):
    buf = kwargs['log_f']
    return np.sum((x != y) * buf[:, np.asarray(x, int)] * buf[:, np.asarray(y, int)])


Для метрики indicator: AUC: 0.829499015542

<img src="auc_indicator.png", width=200, align=left> 


Для метрики smoothed_indicator: AUC: 0.833989009583

<img src="smoothed_indicator.png", width=200, align=left> 


Для метрики log_indicator: AUC: 0.819376603473

<img src="log_indicator.png", width=200, align=left>

#### Лучшие результаты достигнуты на второй метрике

#### 2 (бонус). Подберите лучшее (на тестовой выборке) число соседей $k$ для каждой из функций расстояния. Какое наилучшее качество удалось получить?

Для подбора можно использовать любые средства из sklearn.

# Ответы
#### Весь код можно найти в файле: knn_question_1_and_2

Для подбора использовалась самописная функция find_optimal_k. K перебиралось от 1 до 15.

Для метрики indicator: K = 9, AUC = 0.83094973406339201

Для метрики smoothed_indicator: K = 10, AUC = 0.83469194987351925

Для метрики log_indicator: K = 9, AUC = 0.82167888343285744

## Код

In [ ]:
import pandas as pd
import numpy as np
import copy
from sklearn.neighbors import KNeighborsClassifier, DistanceMetric
import sklearn
import matplotlib.pyplot as plt

data = pd.read_csv('amazon.csv')

max_value = 0
for col_name in data.columns:
    max_value = max(max_value, np.max(data[col_name]))
print(max_value)

from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data.iloc[:, 1:], data.iloc[:, 0],
                                                    test_size=0.3, random_state=241)

def display_auc(y_true, y_predict):
    print("AUC:", sklearn.metrics.roc_auc_score(y_score=y_predict, y_true=y_true))
    (fpr, tpr, thresholds) = sklearn.metrics.roc_curve(y_true=y_true, y_score=y_predict)
    plt.plot(fpr, tpr)
    plt.show()

def get_auc(y_true, y_predict):
    return sklearn.metrics.roc_auc_score(y_score=y_predict, y_true=y_true)

In [ ]:
neigh_with_indicator = sklearn.neighbors.KNeighborsClassifier(n_neighbors=10, algorithm='brute', metric=indicator)
neigh_with_indicator.fit(X_train, y_train)
y_with_indicator = neigh_with_indicator.predict_proba(X_test)

y_with_indicator = y_with_indicator[:, 1]

display_auc(y_true = y_test, y_predict = y_with_indicator)

In [ ]:
def unique_with_count(input_array): # (value, count)
    buf = copy.deepcopy(input_array)
    buf = buf[buf == buf]
    buf = np.sort(buf)
    
    Y = np.concatenate((buf[1:], np.array([np.nan])))
    lens_seg = np.cumsum(np.ones(buf.shape[0]))
    val = buf[buf != Y]
    lens_seg = lens_seg[buf != Y]
    lens_seg = np.asarray(np.concatenate((lens_seg[0:1], np.diff(lens_seg))), dtype = int)
    return (val, lens_seg)


p_2 = np.zeros((X_train.shape[1], max_value + 1))
p = np.zeros((X_train.shape[1], max_value + 1))
f = np.zeros((X_train.shape[1], max_value + 1))
for col in range(X_train.shape[1]):
    (unique_value, count) = unique_with_count(X_train.iloc[:, col])
    cur_f = count
    cur_p = (count + 0.0) / X_train.shape[0]
    cur_p_2 = count * (count - 1.0) / X_train.shape[0] / (X_train.shape[0] - 1)
    sum_p_2 = np.zeros(unique_value.shape[0])
    for i in range(unique_value.shape[0]):
        sum_p_2[i] = np.sum(cur_p_2[cur_p < cur_p[i]])
    
    f[col][unique_value] = cur_f
    p[col][unique_value] = cur_p
    p_2[col][unique_value] = sum_p_2
    
log_f = np.log(f + 1)

In [ ]:
def smoothed_indicator(x, y, **kwargs):
    buf = kwargs['p_2']
    return np.sum((x != y) + (x == y) * buf[:, np.asarray(x, int)])


In [ ]:
#smoothed_indicator
neigh_with_smoothed_indicator = sklearn.neighbors.KNeighborsClassifier(n_neighbors=10, algorithm='brute', metric=smoothed_indicator, 
                                                                       metric_params={'p_2': p_2})
neigh_with_smoothed_indicator.fit(X_train, y_train)


y_with_smoothed_indicator = neigh_with_smoothed_indicator.predict_proba(np.asarray(X_test, int))
y_with_smoothed_indicator = y_with_smoothed_indicator[:, 1]

display_auc(y_true = y_test, y_predict = y_with_smoothed_indicator)

In [ ]:
def log_indicator(x, y, **kwargs):
    buf = kwargs['log_f']
    return np.sum((x != y) * buf[:, np.asarray(x, int)] * buf[:, np.asarray(y, int)])

In [ ]:
#log_indicator
neigh_with_log_indicator = sklearn.neighbors.KNeighborsClassifier(n_neighbors=10, algorithm='brute', 
                                                              metric=log_indicator,
                                                             metric_params={'log_f': log_f})
neigh_with_log_indicator.fit(X_train, y_train)
y_with_log_indicator = neigh_with_log_indicator.predict_proba(np.asarray(X_test, int))

y_with_log_indicator = y_with_log_indicator[:, 1]
display_auc(y_true = y_test, y_predict = y_with_log_indicator)

In [ ]:
def find_optimal_k(X_train, y_train, X_test, y_test, MAXK, metric, metric_params = None):
    MAXK += 1
    classifier = sklearn.neighbors.KNeighborsClassifier(n_neighbors=MAXK, 
                                                        algorithm='brute', 
                                                        metric=metric,
                                                        metric_params=metric_params)
    classifier.fit(X_train, y_train)
    nearest_class = classifier.kneighbors(X_test, return_distance=0)
    precision = np.ndarray((MAXK, X_test.shape[0], 2))
    for ind_object in range(X_test.shape[0]):
        histohram = np.zeros(2)
        for k in range(0, MAXK):
            histohram[y_train.iloc[nearest_class[ind_object][k]]] += 1
            precision[k, ind_object] = histohram / (k + 1)
    
    max_auc = 0.0
    max_k = 0;
    for k in range(0, MAXK):
        y_predict = precision[k, :, 1]
        cur_auc = get_auc(y_true = y_test, y_predict = y_predict)
        if (cur_auc > max_auc):
            max_auc = cur_auc
            max_k = k + 1
        
    return (max_auc, max_k)

In [ ]:
print( find_optimal_k(X_train, y_train, X_test, y_test, 15, log_indicator, metric_params={'log_f': log_f}) )
print( find_optimal_k(X_train, y_train, X_test, y_test, 15, smoothed_indicator, metric_params={'p_2': p_2}) )
print( find_optimal_k(X_train, y_train, X_test, y_test, 15, indicator) )

---

#### 3. Реализуйте счетчики (http://blogs.technet.com/b/machinelearning/archive/2015/02/17/big-learning-made-easy-with-counts.aspx), которые заменят категориальные признаки на вещественные.

А именно, каждый категориальный признак нужно заменить на три: 
1. Число `counts` объектов в обучающей выборке с таким же значением признака.
2. Число `successes` объектов первого класса ($y = 1$) в обучающей выборке с таким же значением признака.
3. Сглаженное отношение двух предыдущих величин: (`successes` + 1) / (`counts` + 2).

Поскольку признаки, содержащие информацию о целевой переменной, могут привести к переобучению, может оказаться полезным сделать *фолдинг*: разбить обучающую выборку на $n$ частей, и для $i$-й части считать `counts` и `successes` по всем остальным частям. Для тестовой выборки используются счетчики, посчитанные по всей обучающей выборке. Реализуйте и такой вариант. Можно использовать $n = 3$.

#### Посчитайте на тесте AUC-ROC метода $k$ ближайших соседей с евклидовой метрикой для выборки, где категориальные признаки заменены на счетчики. Сравните по AUC-ROC два варианта формирования выборки — с фолдингом и без. Не забудьте подобрать наилучшее число соседей $k$.

# Ответы

#### Весь код можно найти в файле: knn_question_3

Для K = 10, без фолдинга AUC: 0.788718055439

<img src="without_folding_10.png", width=200, align=left>


Для K = 10, c фолдинга AUC: 0.763562263815

<img src="with_folding_10.png", width=200, align=left>

В эксперименте без фолдинга лучший результат был достигнут при k = 14 AUC: 0.79678798609179213 

В эксперименте c фолдинга лучший результат был достигнут при k = 16 AUC: 0.77708562015813154 

## Код

In [ ]:
import pandas as pd
import numpy as np
import copy
from sklearn.neighbors import KNeighborsClassifier, DistanceMetric
import sklearn
import matplotlib.pyplot as plt

data = pd.read_csv('amazon.csv')

max_value = 0
for col_name in data.columns:
    max_value = max(max_value, np.max(data[col_name]))
print(max_value)

from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data.iloc[:, 1:], data.iloc[:, 0],
                                                    test_size=0.3, random_state=241)

def display_auc(y_true, y_predict):
    print("AUC:", sklearn.metrics.roc_auc_score(y_score=y_predict, y_true=y_true))
    (fpr, tpr, thresholds) = sklearn.metrics.roc_curve(y_true=y_true, y_score=y_predict)
    plt.plot(fpr, tpr)
    plt.show()

def get_auc(y_true, y_predict):
    return sklearn.metrics.roc_auc_score(y_score=y_predict, y_true=y_true)

def unique_with_count(input_array): # (value, count)
    buf = copy.deepcopy(input_array)
    buf = buf[buf == buf]
    buf = np.sort(buf)
    
    Y = np.concatenate((buf[1:], np.array([np.nan])))
    lens_seg = np.cumsum(np.ones(buf.shape[0]))
    val = buf[buf != Y]
    lens_seg = lens_seg[buf != Y]
    lens_seg = np.asarray(np.concatenate((lens_seg[0:1], np.diff(lens_seg))), dtype = int)
    return (val, lens_seg)


p_2 = np.zeros((X_train.shape[1], max_value + 1))
p = np.zeros((X_train.shape[1], max_value + 1))
f = np.zeros((X_train.shape[1], max_value + 1))
for col in range(X_train.shape[1]):
    (unique_value, count) = unique_with_count(X_train.iloc[:, col])
    cur_f = count
    cur_p = (count + 0.0) / X_train.shape[0]
    cur_p_2 = count * (count - 1.0) / X_train.shape[0] / (X_train.shape[0] - 1)
    sum_p_2 = np.zeros(unique_value.shape[0])
    for i in range(unique_value.shape[0]):
        sum_p_2[i] = np.sum(cur_p_2[cur_p < cur_p[i]])
    
    f[col][unique_value] = cur_f
    p[col][unique_value] = cur_p
    p_2[col][unique_value] = sum_p_2
    
log_f = np.log(f + 1)

def find_optimal_k(X_train, y_train, X_test, y_test, MAXK, metric, metric_params = None):
    MAXK += 1
    classifier = sklearn.neighbors.KNeighborsClassifier(n_neighbors=MAXK, 
                                                        algorithm='brute', 
                                                        metric=metric,
                                                        metric_params=metric_params)
    classifier.fit(X_train, y_train)
    nearest_class = classifier.kneighbors(X_test, return_distance=0)
    precision = np.ndarray((MAXK, X_test.shape[0], 2))
    for ind_object in range(X_test.shape[0]):
        histohram = np.zeros(2)
        for k in range(0, MAXK):
            histohram[y_train.iloc[nearest_class[ind_object][k]]] += 1
            precision[k, ind_object] = histohram / (k + 1)
    
    max_auc = 0.0
    max_k = 0;
    for k in range(0, MAXK):
        y_predict = precision[k, :, 1]
        cur_auc = get_auc(y_true = y_test, y_predict = y_predict)
        if (cur_auc > max_auc):
            max_auc = cur_auc
            max_k = k + 1
        
    return (max_auc, max_k)

def get_count_and_successed(X_train, y_train):
    count = [{} for i in range(X_train.shape[1])]
    successed = [{} for i in range(X_train.shape[1])]
    for col in range(X_train.shape[1]):
        value, len = unique_with_count(X_train.iloc[:, col])
        
        count[col] = {cur_value: 0 for cur_value in value}
        successed[col] = {cur_value: 0 for cur_value in value}
        for ind in range(X_train.shape[0]):
            count[col][X_train.iloc[ind, col]] += 1
            successed[col][X_train.iloc[ind, col]] += (y_train.iloc[ind] == 1)
        for cur_value in value:
            count[col][cur_value] /= y_train.shape[0]
            successed[col][cur_value] /= y_train.shape[0]
            
    return (count, successed)

def get_new_features(X, count, successed):
    cnt = 0
    new_X = np.ndarray((X.shape[0], 3 * X.shape[1]))
    for i in range(X.shape[0]):
        for j in range(X.shape[1]):
            new_X[i, 3 * j] = count[j].get(X.iloc[i, j], 0)
            new_X[i, 3 * j + 1] = successed[j].get(X.iloc[i, j], 0)
            new_X[i, 3 * j + 2] = (new_X[i, 3 * j + 1] + 1) / (new_X[i, 3 * j] + 2)
            cnt += new_X[i, 3 * j] == 0;
    print(cnt)
    return new_X

def get_new_features_with_fold(X, y, CNT_FOLDS = 3):
    new_X = np.ndarray((X.shape[0], 3 * X.shape[1]))
    for i in range(CNT_FOLDS):
        ind = np.ndarray(((X.shape[0] + CNT_FOLDS - 1 - i) // CNT_FOLDS), dtype=int)
        other_ind = np.ndarray(X.shape[0] - ind.shape[0], dtype=int)
        cnt = 0
        cnt_other = 0
        for j in range(X.shape[0]):
            if (j % CNT_FOLDS == i):
                ind[cnt] = j
                cnt += 1
            else:
                other_ind[cnt_other] = j
                cnt_other += 1

        count, successed = get_count_and_successed(X.iloc[other_ind, :], y.iloc[other_ind])
        X_chunk_new = get_new_features(X.iloc[ind, :], count, successed)
        new_X[ind, :] = X_chunk_new
    return new_X


In [ ]:
count, successed = get_count_and_successed(X_train, y_train)
new_X_train = get_new_features(X_train, count, successed)
new_X_test = get_new_features(X_test, count, successed)

In [ ]:
#WITHOUT FOLDING
neigh = sklearn.neighbors.KNeighborsClassifier(n_neighbors=10, 
                                               algorithm='brute', 
                                               metric='euclidean')

neigh.fit(new_X_train, y_train)


y = neigh.predict_proba(new_X_test)
y = y[:, 1]

display_auc(y_true = y_test, y_predict = y)

In [ ]:
find_optimal_k(new_X_train, y_train, new_X_test, y_test, MAXK=15, metric='euclidean') #WITHOUT FOLDING

In [ ]:
#WITH FOLDING
CNT_FOLDS = 3
new_X_train = get_new_features_with_fold(X_train, y_train, CNT_FOLDS)


count, successed = get_count_and_successed(X_train, y_train)
new_X_test = get_new_features(X_test, count, successed)


neigh = sklearn.neighbors.KNeighborsClassifier(n_neighbors=10, 
                                               algorithm='brute', 
                                               metric='euclidean')

neigh.fit(new_X_train, y_train)


y = neigh.predict_proba(new_X_test)
y = y[:, 1]

display_auc(y_true = y_test, y_predict = y)

In [ ]:
find_optimal_k(new_X_train, y_train, new_X_test, y_test, MAXK=30, metric='euclidean') #WITH FOLDING

---

#### 4. Добавьте в исходную выборку парные признаки — то есть для каждой пары $(f_i, f_j)$, $i < j$ исходных категориальных признаков добавьте новый категориальный признак $f_{ij}$, значение которого является конкатенацией значений $f_i$ и $f_j$ (желательно через какой-нибудь специальный символ во избежание коллизий). Посчитайте счетчики для этой выборки, найдите качество метода $k$ ближайших соседей с наилучшим $k$ (с фолдингом и без).

# Ответы

Эксперимент с фолдингом:


При К = 10 AUC: 0.786943587222

<img src="with_folding_with_pair_features.png", width=200, align=left>


Эксперимент без фолдинга:

При К = 10 AUC: 0.80338239808

<img src="without_folding_with_pair_features.png", width=200, align=left>

## Код

In [ ]:
import pandas as pd
import numpy as np
import copy
from sklearn.neighbors import KNeighborsClassifier, DistanceMetric
import sklearn
import matplotlib.pyplot as plt

data = pd.read_csv('amazon.csv')
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data.iloc[:, 1:], data.iloc[:, 0],
                                                    test_size=0.3, random_state=241)

max_value = 0
for col_name in data.columns:
    max_value = max(max_value, np.max(data[col_name]))
print(max_value)

def get_pair_features(X):
    print("Begin")
    X_new = np.ndarray((X.shape[0], X.shape[1] * (X.shape[1] + 1) // 2))
    X_new[:, 0:X.shape[1]] = copy.deepcopy(X.iloc[:])
    print("End copy")
    len = X.shape[1]
    for i in range(X.shape[1]):
        print("Processing", i, "features")
        for j in range(i + 1, X.shape[1]):
            for k in range(X.shape[0]):
                X_new[k, len] = str(X.iloc[k, i]) + "0123321" + str(X.iloc[k, j])
            len += 1
    return X_new

In [ ]:
X_train_with_pair_features = get_pair_features(X_train)
X_test_with_pair_features = get_pair_features(X_test)

In [ ]:
def get_new_features(X, count, successed):
    cnt = 0
    new_X = np.ndarray((X.shape[0], 3 * X.shape[1]))
    for i in range(X.shape[0]):
        for j in range(X.shape[1]):
            new_X[i, 3 * j] = count[j].get(X[i, j], 0)
            new_X[i, 3 * j + 1] = successed[j].get(X[i, j], 0)
            new_X[i, 3 * j + 2] = (new_X[i, 3 * j + 1] + 1) / (new_X[i, 3 * j] + 2)
            cnt += new_X[i, 3 * j] == 0;
    print(cnt)
    return new_X

def get_count_and_successed(X_train, y_train):
    count = [{} for i in range(X_train.shape[1])]
    successed = [{} for i in range(X_train.shape[1])]
    for col in range(X_train.shape[1]):
        value, len = unique_with_count(X_train[:, col])
        
        count[col] = {cur_value: 0 for cur_value in value}
        successed[col] = {cur_value: 0 for cur_value in value}
        for ind in range(X_train.shape[0]):
            count[col][X_train[ind, col]] += 1
            successed[col][X_train[ind, col]] += (y_train[ind] == 1)
        for cur_value in value:
            count[col][cur_value] /= y_train.shape[0]
            successed[col][cur_value] /= y_train.shape[0]
            
    return (count, successed)

def unique_with_count(input_array): # (value, count)
    buf = copy.deepcopy(input_array)
    buf = buf[buf == buf]
    buf = np.sort(buf)
    
    Y = np.concatenate((buf[1:], np.array([np.nan])))
    lens_seg = np.cumsum(np.ones(buf.shape[0]))
    val = buf[buf != Y]
    lens_seg = lens_seg[buf != Y]
    lens_seg = np.asarray(np.concatenate((lens_seg[0:1], np.diff(lens_seg))), dtype = int)
    return (val, lens_seg)


p_2 = np.zeros((X_train.shape[1], max_value + 1))
p = np.zeros((X_train.shape[1], max_value + 1))
f = np.zeros((X_train.shape[1], max_value + 1))
for col in range(X_train.shape[1]):
    (unique_value, count) = unique_with_count(X_train.iloc[:, col])
    cur_f = count
    cur_p = (count + 0.0) / X_train.shape[0]
    cur_p_2 = count * (count - 1.0) / X_train.shape[0] / (X_train.shape[0] - 1)
    sum_p_2 = np.zeros(unique_value.shape[0])
    for i in range(unique_value.shape[0]):
        sum_p_2[i] = np.sum(cur_p_2[cur_p < cur_p[i]])
    
    f[col][unique_value] = cur_f
    p[col][unique_value] = cur_p
    p_2[col][unique_value] = sum_p_2
    
log_f = np.log(f + 1)

def display_auc(y_true, y_predict):
    print("AUC:", sklearn.metrics.roc_auc_score(y_score=y_predict, y_true=y_true))
    (fpr, tpr, thresholds) = sklearn.metrics.roc_curve(y_true=y_true, y_score=y_predict)
    plt.plot(fpr, tpr)
    plt.show()

def get_auc(y_true, y_predict):
    return sklearn.metrics.roc_auc_score(y_score=y_predict, y_true=y_true)

In [ ]:
count, successed = get_count_and_successed(X_train_with_pair_features, np.array(y_train.iloc[:], dtype=int))
new_X_train_with_pair_features = get_new_features(X_train_with_pair_features, count, successed)
new_X_test_with_pair_features = get_new_features(X_test_with_pair_features, count, successed)

In [ ]:
def find_optimal_k(X_train, y_train, X_test, y_test, MAXK, metric, metric_params = None):
    MAXK += 1
    classifier = sklearn.neighbors.KNeighborsClassifier(n_neighbors=MAXK, 
                                                        algorithm='brute', 
                                                        metric=metric,
                                                        metric_params=metric_params)
    classifier.fit(X_train, y_train)
    nearest_class = classifier.kneighbors(X_test, return_distance=0)
    precision = np.ndarray((MAXK, X_test.shape[0], 2))
    for ind_object in range(X_test.shape[0]):
        histohram = np.zeros(2)
        for k in range(0, MAXK):
            histohram[y_train.iloc[nearest_class[ind_object][k]]] += 1
            precision[k, ind_object] = histohram / (k + 1)
    
    max_auc = 0.0
    max_k = 0;
    max_y = None
    for k in range(0, MAXK):
        y_predict = precision[k, :, 1]
        cur_auc = get_auc(y_true = y_test, y_predict = y_predict)
        if (cur_auc > max_auc):
            max_auc = cur_auc
            max_k = k + 1
            max_y = y_predict
            
    display_auc(y_true=y_test, y_predict=max_y)
    return (max_auc, max_k)

In [ ]:
#WITHOUT FOLDING
print(find_optimal_k(new_X_train_with_pair_features, y_train, new_X_test_with_pair_features, y_test, 30, 'euclidean'))


In [ ]:
#WITH FOLDING
def get_new_features_with_fold(X, y, CNT_FOLDS = 3):
    new_X = np.ndarray((X.shape[0], 3 * X.shape[1]))
    for i in range(CNT_FOLDS):
        ind = np.ndarray(((X.shape[0] + CNT_FOLDS - 1 - i) // CNT_FOLDS), dtype=int)
        other_ind = np.ndarray(X.shape[0] - ind.shape[0], dtype=int)
        cnt = 0
        cnt_other = 0
        for j in range(X.shape[0]):
            if (j % CNT_FOLDS == i):
                ind[cnt] = j
                cnt += 1
            else:
                other_ind[cnt_other] = j
                cnt_other += 1

        count, successed = get_count_and_successed(X[other_ind, :], y[other_ind])
        X_chunk_new = get_new_features(X[ind, :], count, successed)
        new_X[ind, :] = X_chunk_new
    return new_X

CNT_FOLDS = 3
new_X_train_with_pair_features = get_new_features_with_fold(X_train_with_pair_features, np.array(y_train, dtype=int), CNT_FOLDS)


count, successed = get_count_and_successed(X_train_with_pair_features, np.array(y_train, dtype=int))
new_X_test_with_pair_features = get_new_features(X_test_with_pair_features, count, successed)


print(find_optimal_k(new_X_train_with_pair_features, y_train, new_X_test_with_pair_features, y_test, 30, 'euclidean'))


---

## Часть 3: Решающие деревья и леса

#### 1. Возьмите из предыдущей части выборку с парными признаками, преобразованную с помощью счетчиков без фолдинга. Настройте решающее дерево, подобрав оптимальные значения параметров `max_depth` и `min_samples_leaf`. Какой наилучший AUC-ROC на контроле удалось получить?

# Ответы
Весь код можно найти в файле: tree_question_1_and_2

Оптимальные параметры:

    max_depth = 20
    
    min_samples_leaf = 15
    
AUC: 0.780350343356

<img src="tree.png", width=200, align=left>

#### 2. Настройте случайный лес, подобрав такое число деревьев `n_estimators`, при котором ошибка выходит на асимптоту. Какое качество на тестовой выборке он дает?

# Ответы
Весь код можно найти в файле: tree_question_1_and_2

Эксперимент проводились при n_estimators = 1000

AUC: 0.847733141114

<img src="forest.png", width=200, align=left>


#### 3. Возьмите выборку с парными признаками, для которой счетчики посчитаны с фолдингом. Обучите на ней случайный лес, подобрав число деревьев. Какое качество на тестовой выборке он дает? Чем вы можете объяснить изменение результата по сравнению с предыдущим пунктом?

# Ответы
Весь код можно найти в файле: tree_question_3

Эксперимент проводились при n_estimators = 1000

AUC: 0.847844202625

<img src="forest_with_folding.png", width=200, align=left>

При фолдинге небольшое преимущество, это связано с тем что фолдинг уменьшает вероятность переобучения. Оно такое незначительное, потому что лес также уменьшает вероятность переобучения.

## Код

In [ ]:
import pandas as pd
import numpy as np
import copy
from sklearn.neighbors import KNeighborsClassifier, DistanceMetric
import sklearn
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

data = pd.read_csv('amazon.csv')
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data.iloc[:, 1:], data.iloc[:, 0],
                                                    test_size=0.3, random_state=241)

max_value = 0
for col_name in data.columns:
    max_value = max(max_value, np.max(data[col_name]))
print(max_value)

def get_pair_features(X):
    print("Begin")
    X_new = np.ndarray((X.shape[0], X.shape[1] * (X.shape[1] + 1) // 2))
    X_new[:, 0:X.shape[1]] = copy.deepcopy(X.iloc[:])
    print("End copy")
    len = X.shape[1]
    for i in range(X.shape[1]):
        print("Processing", i, "features")
        for j in range(i + 1, X.shape[1]):
            X_new[:, len] = X.iloc[:, i] * 1000 * 1000 * 1000 + X.iloc[:, j] 
            len += 1
    return X_new

In [ ]:
X_train_with_pair_features = get_pair_features(X_train)
X_test_with_pair_features = get_pair_features(X_test)

In [ ]:
def get_new_features(X, count, successed):
    cnt = 0
    new_X = np.ndarray((X.shape[0], 3 * X.shape[1]))
    for i in range(X.shape[0]):
        for j in range(X.shape[1]):
            new_X[i, 3 * j] = count[j].get(X[i, j], 0)
            new_X[i, 3 * j + 1] = successed[j].get(X[i, j], 0)
            new_X[i, 3 * j + 2] =  round((new_X[i, 3 * j + 1] + 1) / (new_X[i, 3 * j] + 2), 4)
            cnt += new_X[i, 3 * j] == 0;
    print(cnt)
    return new_X

def get_count_and_successed(X_train, y_train):
    count = [{} for i in range(X_train.shape[1])]
    successed = [{} for i in range(X_train.shape[1])]
    for col in range(X_train.shape[1]):
        value, len = unique_with_count(X_train[:, col])
        
        count[col] = {cur_value: 0 for cur_value in value}
        successed[col] = {cur_value: 0 for cur_value in value}
        for ind in range(X_train.shape[0]):
            count[col][X_train[ind, col]] += 1
            successed[col][X_train[ind, col]] += (y_train[ind] == 1)
        
        for cur_value in value:
            count[col][cur_value] /= y_train.shape[0]
            successed[col][cur_value] /= y_train.shape[0]
            successed[col][cur_value] = round(successed[col][cur_value], 3)
    return (count, successed)

def unique_with_count(input_array): # (value, count)
    buf = copy.deepcopy(input_array)
    buf = buf[buf == buf]
    buf = np.sort(buf)
    
    Y = np.concatenate((buf[1:], np.array([np.nan])))
    lens_seg = np.cumsum(np.ones(buf.shape[0]))
    val = buf[buf != Y]
    lens_seg = lens_seg[buf != Y]
    lens_seg = np.asarray(np.concatenate((lens_seg[0:1], np.diff(lens_seg))), dtype = int)
    return (val, lens_seg)

def display_auc(y_true, y_predict):
    print("AUC:", sklearn.metrics.roc_auc_score(y_score=y_predict, y_true=y_true))
    (fpr, tpr, thresholds) = sklearn.metrics.roc_curve(y_true=y_true, y_score=y_predict)
    plt.plot(fpr, tpr)
    plt.show()

def get_auc(y_true, y_predict):
    return sklearn.metrics.roc_auc_score(y_score=y_predict, y_true=y_true)

In [ ]:
count, successed = get_count_and_successed(X_train_with_pair_features, np.array(y_train.iloc[:], dtype=int))
new_X_train_with_pair_features = get_new_features(X_train_with_pair_features, count, successed)
new_X_test_with_pair_features = get_new_features(X_test_with_pair_features, count, successed)

In [ ]:
max_auc = 0
max_max_depth = 0
max_min_samples_leaf = 0
max_y = None
for max_depth in range(20, 30, 2):
    for min_samples_leaf in range(10, 100, 5):
        classifier = sklearn.tree.DecisionTreeClassifier(max_depth=max_depth, min_samples_leaf=min_samples_leaf )
        classifier.fit(new_X_train_with_pair_features, np.array(y_train.iloc[:], dtype=int))
        y = classifier.predict_proba(new_X_test_with_pair_features)
        y = y[:, 1]

        cur_auc = get_auc(y_true= y_test, y_predict = y)
        if (max_auc < cur_auc):
            max_auc = cur_auc
            max_max_depth = max_depth
            max_min_samples_leaf = min_samples_leaf
            max_y = y
        
print(max_auc, max_max_depth, max_min_samples_leaf)
display_auc(y_predict= max_y, y_true=y_test)

In [ ]:
classifier = sklearn.ensemble.RandomForestClassifier(n_estimators=1000, max_depth=20, min_samples_leaf=15)
classifier.fit(new_X_train_with_pair_features, y_train)
y = classifier.predict_proba(new_X_test_with_pair_features)
y = y[:, 1]
display_auc(y_true=y_test, y_predict=y)

---

Здесь вы можете поделиться своими мыслями о задании.

С дотой задание прикольнее ;(

А здесь вставьте смешную картинку.

А здесь посоветуйте преподавателям хороший фильм или сериал.

Хранители